### Import Library

In [ ]:
# 이 중에서 안깔린 라이브러리가 있을 수 있음

import os
from typing import List, Dict
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
import lightgbm as lgb
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import seaborn as sns
import matplotlib.pyplot as plt
import pywt



### Load Data

In [2]:
path = "../data/"

train = pd.read_csv(path+"train.csv").assign(_type="train")
test = pd.read_csv(path+"test.csv").assign(_type="test")
submission = pd.read_csv(path+"test.csv")
df = pd.concat([train, test], axis=0)

##### 원래 베이스라인 코드 부분

In [ ]:
# # HOURLY_ 로 시작하는 .csv 파일 이름을 file_names 에 할딩
# file_names: List[str] = [
#     f for f in os.listdir(data_path) if f.startswith("HOURLY_") and f.endswith(".csv")
# ]

# # 파일명 : 데이터프레임으로 딕셔너리 형태로 저장
# file_dict: Dict[str, pd.DataFrame] = {
#     f.replace(".csv", ""): pd.read_csv(os.path.join(data_path, f)) for f in file_names
# }

# for _file_name, _df in tqdm(file_dict.items()):
#     # 열 이름 중복 방지를 위해 {_file_name.lower()}_{col.lower()}로 변경, datetime 열을 ID로 변경
#     _rename_rule = {
#         col: f"{_file_name.lower()}_{col.lower()}" if col != "datetime" else "ID"
#         for col in _df.columns
#     }
#     _df = _df.rename(_rename_rule, axis=1)
#     df = df.merge(_df, on="ID", how="left")

##### 밑의 한 셀이 바꾼 부분 (이 바꾼 부분을 전제로 뒤의 코드가 실행됨.)

In [3]:
# HOURLY_로 시작하는 모든 csv 파일 불러오기
prefixes = ["HOURLY_"]

file_names = [
    f for f in os.listdir(path)
    if any(f.startswith(prefix) for prefix in prefixes) and f.endswith(".csv")
]
display(file_names)
file_dict: Dict[str, pd.DataFrame] = {
    f.replace(".csv", ""): pd.read_csv(path+f) for f in file_names
}


for _file_name, _df in tqdm(file_dict.items()):
    _rename_rule = {
        col: f"{col.lower()}" if col != "datetime" else "ID"
        for col in _df.columns
    }
    _df = _df.rename(_rename_rule, axis=1)
    df = df.merge(_df, on="ID", how="left")


['HOURLY_NETWORK-DATA_TRANSACTIONS-COUNT.csv',
 'HOURLY_MARKET-DATA_TAKER-BUY-SELL-STATS_ALL_EXCHANGE.csv',
 'HOURLY_NETWORK-DATA_BLOCK-INTERVAL.csv',
 'HOURLY_NETWORK-DATA_ADDRESSES-COUNT.csv',
 'HOURLY_MARKET-DATA_COINBASE-PREMIUM-INDEX.csv',
 'HOURLY_MARKET-DATA_OPEN-INTEREST_ALL_EXCHANGE_ALL_SYMBOL.csv',
 'HOURLY_NETWORK-DATA_HASHRATE.csv',
 'HOURLY_NETWORK-DATA_BLOCKREWARD.csv',
 'HOURLY_NETWORK-DATA_SUPPLY.csv',
 'HOURLY_MARKET-DATA_FUNDING-RATES_ALL_EXCHANGE.csv',
 'HOURLY_MARKET-DATA_PRICE-OHLCV_ALL_EXCHANGE_SPOT_BTC_USD.csv',
 'HOURLY_NETWORK-DATA_BLOCK-COUNT.csv',
 'HOURLY_NETWORK-DATA_UTXO-COUNT.csv',
 'HOURLY_NETWORK-DATA_BLOCK-BYTES.csv',
 'HOURLY_NETWORK-DATA_FEES-TRANSACTION.csv',
 'HOURLY_NETWORK-DATA_TOKENS-TRANSFERRED.csv',
 'HOURLY_NETWORK-DATA_FEES.csv',
 'HOURLY_NETWORK-DATA_VELOCITY.csv',
 'HOURLY_NETWORK-DATA_DIFFICULTY.csv',
 'HOURLY_MARKET-DATA_LIQUIDATIONS_ALL_EXCHANGE_ALL_SYMBOL.csv']

100%|██████████| 20/20 [00:00<00:00, 31.07it/s]


##### 결측치 (아거도 추가된 부분)

In [4]:
df_new = df.drop(["ID", "target", "_type"], axis = 1)

# 결측치 확인
missing_data = df_new.isnull().sum()

# 결측치 비율 확인
missing_ratio = df_new.isnull().mean()


display(missing_data.sort_values(ascending=False))
display(missing_ratio.sort_values(ascending=False))

transactions_count_total          4
transactions_count_mean          28
taker_buy_volume                 31
taker_sell_volume                31
taker_buy_ratio                  31
taker_sell_ratio                 31
taker_buy_sell_ratio             31
block_interval                   24
addresses_count_active            0
addresses_count_sender            0
addresses_count_receiver          0
coinbase_premium_gap              6
coinbase_premium_index            6
open_interest                     4
hashrate                          0
blockreward                       0
blockreward_usd                   0
supply_total                     28
supply_new                       28
funding_rates                    32
close                          2792
volume                         2792
block_count                       4
utxo_count                        1
block_bytes                      54
fees_transaction_mean            24
fees_transaction_mean_usd        24
fees_transaction_median     

transactions_count_total       0.000346
transactions_count_mean        0.002424
taker_buy_volume               0.002684
taker_sell_volume              0.002684
taker_buy_ratio                0.002684
taker_sell_ratio               0.002684
taker_buy_sell_ratio           0.002684
block_interval                 0.002078
addresses_count_active         0.000000
addresses_count_sender         0.000000
addresses_count_receiver       0.000000
coinbase_premium_gap           0.000519
coinbase_premium_index         0.000519
open_interest                  0.000346
hashrate                       0.000000
blockreward                    0.000000
blockreward_usd                0.000000
supply_total                   0.002424
supply_new                     0.002424
funding_rates                  0.002770
close                          0.241690
volume                         0.241690
block_count                    0.000346
utxo_count                     0.000087
block_bytes                    0.004675


In [5]:
# 1. 결측치 처리 - 앞/뒤 보간법 (forward/backward fill)

# df_ffill = df_new.fillna(method='ffill')  # 앞의 값을 사용하여 결측치 채움
# df_bfill = df_new.fillna(method='bfill')  # 뒤의 값을 사용하여 결측치 채움

# 2. 결측치 처리 - 선형 보간법 (linear interpolation)
df_interpolated = df_new.interpolate(method='linear')

# 3. 결측치 처리 - 평균값으로 대체
# df_mean_filled = df_new.fillna(eda_df.mean())

df.update(df_interpolated)

# 결측치 처리 후 확인
print(df_interpolated.isnull().sum())

transactions_count_total       0
transactions_count_mean        0
taker_buy_volume               0
taker_sell_volume              0
taker_buy_ratio                0
taker_sell_ratio               0
taker_buy_sell_ratio           0
block_interval                 0
addresses_count_active         0
addresses_count_sender         0
addresses_count_receiver       0
coinbase_premium_gap           0
coinbase_premium_index         0
open_interest                  0
hashrate                       0
blockreward                    0
blockreward_usd                0
supply_total                   0
supply_new                     0
funding_rates                  0
close                          0
volume                         0
block_count                    0
utxo_count                     0
block_bytes                    0
fees_transaction_mean          0
fees_transaction_mean_usd      0
fees_transaction_median        0
fees_transaction_median_usd    0
tokens_transferred_total       0
tokens_tra

### Feature engineering

만약 데이터를 로드할 때, (제공 받은)베이스라인 코드에서와 같이 거래소와 암호화폐 종류를 불러오도록 설정을 했다면

아래 코드는 작동하지 않음

거래소와 암호화폐 종류를 고려하지 않은 코드임을 유의.

(수정을 해야할 것 같음)

In [6]:
# 전체 독립변수에서 내가 사용할 독립변수 고르기


# 전체 feature
features = df.columns
# 내가 쓸 feature
fe = ['ID', 'target', '_type', 'funding_rates', 'open_interest',
       'taker_buy_volume', 'taker_sell_volume', 'taker_buy_ratio',
       'taker_sell_ratio', 'taker_buy_sell_ratio', 'difficulty',
       'transactions_count_total', 'transactions_count_mean', 'block_count',
       'fees_transaction_mean', 'fees_transaction_mean_usd',
       'fees_transaction_median', 'fees_transaction_median_usd',
       'fees_block_mean', 'fees_block_mean_usd', 'fees_total',
       'fees_total_usd', 'fees_reward_percent', 'hashrate', 'utxo_count',
       'tokens_transferred_total', 'tokens_transferred_mean',
       'tokens_transferred_median', 'block_interval', 'velocity_supply_total',
       'supply_total', 'supply_new', 'addresses_count_active',
       'addresses_count_sender', 'addresses_count_receiver',
       'blockreward', 'blockreward_usd', 'buy_sell_volume_ratio']

# new_features에는 내가 사용하지 않을 feature들이 담기고 나중에 drop됨. 때문에, 이름이 좀 잘못된 듯
new_features = [x for x in features if x not in fe]

In [7]:
# eda 에서 파악한 차이와 차이의 음수, 양수 여부를 새로운 피쳐로 생성
df = df.assign(
    liquidation_diff=df["long_liquidations"] - df["short_liquidations"],
    liquidation_usd_diff=df["long_liquidations_usd"] - df["short_liquidations_usd"],
    volume_diff=df["taker_buy_volume"] - df["taker_sell_volume"],
    liquidation_diffg=np.sign(df["long_liquidations"] - df["short_liquidations"]),
    liquidation_usd_diffg=np.sign(df["long_liquidations_usd"] - df["short_liquidations_usd"]),
    volume_diffg=np.sign(df["taker_buy_volume"] - df["taker_sell_volume"]),
    buy_sell_volume_ratio=df["taker_buy_volume"] / (df["taker_sell_volume"] + 1),
)

# category, continuous 열을 따로 할당해둠
category_cols = ["liquidation_diffg", "liquidation_usd_diffg", "volume_diffg"]
conti_cols = [_ for _ in (df.columns)[3:] if _ not in ["ID", "target", "_type"]] + [
    "buy_sell_volume_ratio",
    "liquidation_diff",
    "liquidation_usd_diff",
    "volume_diff",
]

In [8]:
def shift_feature(
    df: pd.DataFrame,
    conti_cols: List[str],
    intervals: List[int],
) -> List[pd.Series]:
    """
    연속형 변수의 shift feature 생성
    Args:
        df (pd.DataFrame)
        conti_cols (List[str]): continuous colnames
        intervals (List[int]): shifted intervals
    Return:
        List[pd.Series]
    """
    df_shift_dict = [
        df[conti_col].shift(interval).rename(f"{conti_col}_{interval}")
        for conti_col in conti_cols
        for interval in intervals
    ]
    return df_shift_dict

# 지수 이동 평균
def EMA(df, col, span=2):
    return df[col].ewm(span=span).mean()

# Wavlet Transform
def WT(df, col, wavelet='db5', th=0.6):
    signal = df[col].values
    th = th*np.nanmax(signal)
    coef = pywt.wavedec(signal, wavelet, mode="per" )
    coef[1:] = (pywt.threshold(i, value=th, mode="soft" ) for i in coef[1:])
    reconstructed = pywt.waverec(coef, wavelet, mode="per" )
    return reconstructed

In [9]:
# 위의 함수들을 실행한다.

# 모든 수치형 컬럼에 대한 지수이동평균 계산해서 새로운 열로 할당
for c in conti_cols:
    df[c+"_moving_avg_7"] = EMA(df, c, 7)

# 모든 수치형 컬럼에 대한 Wavelet transform을 계산해서 새로운 열로 할당
for c in conti_cols:
    df[c+"WT"] = WT(df, c)

# 최대 24시간의 shift 피쳐를 계산
shift_list = shift_feature(
    df=df, conti_cols=conti_cols, intervals=[_ for _ in range(1, 24)]
)

/tmp/ipykernel_253044/201995820.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[c+"WT"] = WT(df, c)
/tmp/ipykernel_253044/201995820.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[c+"WT"] = WT(df, c)
/tmp/ipykernel_253044/201995820.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[

In [10]:
df = pd.concat([df, pd.concat(shift_list, axis=1)], axis=1)

_target = df["target"]
df = df.ffill().fillna(-999).assign(target = _target)
df = df.loc[:, ~df.columns.duplicated()]
df = df.drop(columns=new_features)

train_df = df.loc[df["_type"]=="train"].drop(columns=["_type"])
test_df = df.loc[df["_type"]=="test"].drop(columns=["_type"])

### Model Training

In [ ]:
# train_test_split 으로 valid set, train set 분리
x_train, x_valid, y_train, y_valid = train_test_split(
    train_df.drop(["target", "ID"], axis = 1), 
    train_df["target"].astype(int), 
    test_size=0.2,
    random_state=42,
)

# lgb dataset
train_data = lgb.Dataset(x_train, label=y_train)
valid_data = lgb.Dataset(x_valid, label=y_valid, reference=train_data)

# lgb params (베이스라인 코드의 기본 파라미터임)
params = {
    "boosting_type": "gbdt",
    "objective": "multiclass",
    "metric": "multi_logloss",
    "num_class": 4,
    "num_leaves": 50,
    "learning_rate": 0.05,
    "n_estimators": 30,
    "random_state": 42,
    "verbose": 0,
}

# lgb train
lgb_model = lgb.train(
    params=params,
    train_set=train_data,
    valid_sets=valid_data,
)

# lgb predict
y_valid_pred = lgb_model.predict(x_valid)
y_valid_pred_class = np.argmax(y_valid_pred, axis = 1)

# score check
accuracy = accuracy_score(y_valid, y_valid_pred_class)
auroc = roc_auc_score(y_valid, y_valid_pred, multi_class="ovr")

print(f"acc: {accuracy}, auroc: {auroc}")

/opt/anaconda3/envs/ml-lecture/lib/python3.9/site-packages/lightgbm/engine.py:172: UserWarning:

Found `n_estimators` in params. Will use it instead of argument



acc: 0.4223744292237443, auroc: 0.6145133857950079


In [58]:
# performance 체크후 전체 학습 데이터로 다시 재학습
x_train = train_df.drop(["target", "ID"], axis = 1)
y_train = train_df["target"].astype(int)
train_data = lgb.Dataset(x_train, label=y_train)
lgb_model = lgb.train(
    params=params,
    train_set=train_data,
)

/opt/conda/lib/python3.10/site-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


### Inference

In [59]:
y_test_pred = lgb_model.predict(test_df.drop(["target", "ID"], axis = 1))
y_test_pred_class = np.argmax(y_test_pred, axis = 1)

### Output File Save

In [60]:
submission = submission.assign(target = y_test_pred_class)
submission.to_csv("output.csv", index=False)